In [1]:
## This file implements neural networks and random forest on p0040presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 59.77% for over-sampling data, 60.92% after improvement.
## And it is 96.43% for combination data.
## For random forest, the accuracy is 55.17% for over-sampling data and 100% for combination data.
## For random forest with cross-validation, the mean accuracy is 60.77% for over-sampling, and 98.46% for combination data.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0040presabs_qual.csv')
df.shape

(255, 373)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
5      2
6      2
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     2
15     2
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     2
24     2
25     0
26     2
27     0
28     1
29     2
      ..
225    2
226    1
227    1
228    2
229    2
230    1
231    1
232    2
233    1
234    1
235    2
236    1
237    2
238    1
239    0
240    1
241    1
242    0
243    1
244    1
245    2
246    2
247    1
248    2
249    1
250    2
251    1
252    1
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

2    96
1    95
0    64
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 372)

In [9]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 371) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 96), (1, 96), (2, 96)]


Using TensorFlow backend.


In [12]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 44), (1, 11), (2, 37)]


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [17]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/100
201/201 [==============================] - 0s 554us/step - loss: 1.0516 - accuracy: 0.4677 - val_loss: 1.0154 - val_accuracy: 0.4943
Epoch 2/100
201/201 [==============================] - 0s 100us/step - loss: 0.9044 - accuracy: 0.5572 - val_loss: 0.9822 - val_accuracy: 0.4828
Epoch 3/100
201/201 [==============================] - 0s 140us/step - loss: 0.8258 - accuracy: 0.6219 - val_loss: 0.9917 - val_accuracy: 0.4713
Epoch 4/100
201/201 [==============================] - 0s 124us/step - loss: 0.7946 - accuracy: 0.6567 - val_loss: 1.0096 - val_accuracy: 0.5057
Epoch 5/100
201/201 [==============================] - 0s 128us/step - loss: 0.7733 - accuracy: 0.6617 - val_loss: 1.0237 - val_accuracy: 0.4253
Epoch 6/100
201/201 [==============================] - 0s 131us/step - loss: 0.7404 - accuracy: 0.6716 - val_loss: 0.9930 - val_accuracy: 0.5402
Epoch 7/100
201/201 [==============================] - 0s 130us/step - loss: 0.7211 -

Epoch 57/100
201/201 [==============================] - 0s 130us/step - loss: 0.3208 - accuracy: 0.9055 - val_loss: 1.0712 - val_accuracy: 0.6207
Epoch 58/100
201/201 [==============================] - 0s 108us/step - loss: 0.3070 - accuracy: 0.9154 - val_loss: 1.0787 - val_accuracy: 0.6207
Epoch 59/100
201/201 [==============================] - 0s 103us/step - loss: 0.3178 - accuracy: 0.8756 - val_loss: 1.1570 - val_accuracy: 0.5747
Epoch 60/100
201/201 [==============================] - 0s 97us/step - loss: 0.3083 - accuracy: 0.8806 - val_loss: 1.0711 - val_accuracy: 0.6207
Epoch 61/100
201/201 [==============================] - 0s 95us/step - loss: 0.3006 - accuracy: 0.9154 - val_loss: 1.1006 - val_accuracy: 0.6207
Epoch 62/100
201/201 [==============================] - 0s 97us/step - loss: 0.2960 - accuracy: 0.9104 - val_loss: 1.1069 - val_accuracy: 0.5977
Epoch 63/100
201/201 [==============================] - 0s 98us/step - loss: 0.3021 - accuracy: 0.9204 - val_loss: 1.1362 - val

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

87/87 [==============================] - 0s 179us/step
over-sampling test accuracy: 59.77%


In [29]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [30]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/1000
201/201 [==============================] - 0s 670us/step - loss: 1.0752 - accuracy: 0.3881 - val_loss: 1.0163 - val_accuracy: 0.3678
Epoch 2/1000
201/201 [==============================] - 0s 165us/step - loss: 0.9163 - accuracy: 0.5721 - val_loss: 0.9949 - val_accuracy: 0.5287
Epoch 3/1000
201/201 [==============================] - 0s 171us/step - loss: 0.8617 - accuracy: 0.6020 - val_loss: 0.9955 - val_accuracy: 0.5057
Epoch 4/1000
201/201 [==============================] - 0s 169us/step - loss: 0.8187 - accuracy: 0.6269 - val_loss: 0.9882 - val_accuracy: 0.5287
Epoch 5/1000
201/201 [==============================] - 0s 172us/step - loss: 0.7850 - accuracy: 0.6468 - val_loss: 0.9999 - val_accuracy: 0.4598
Epoch 6/1000
201/201 [==============================] - 0s 202us/step - loss: 0.7467 - accuracy: 0.6915 - val_loss: 1.0037 - val_accuracy: 0.5057
Epoch 7/1000
201/201 [==============================] - 0s 173us/step - loss: 0

201/201 [==============================] - 0s 186us/step - loss: 0.1921 - accuracy: 0.9254 - val_loss: 1.6158 - val_accuracy: 0.5057
Epoch 57/1000
201/201 [==============================] - 0s 183us/step - loss: 0.1900 - accuracy: 0.9453 - val_loss: 1.6460 - val_accuracy: 0.5287
Epoch 58/1000
201/201 [==============================] - 0s 148us/step - loss: 0.1875 - accuracy: 0.9303 - val_loss: 1.5799 - val_accuracy: 0.5747
Epoch 59/1000
201/201 [==============================] - 0s 142us/step - loss: 0.1780 - accuracy: 0.9453 - val_loss: 1.6553 - val_accuracy: 0.5517
Epoch 60/1000
201/201 [==============================] - 0s 149us/step - loss: 0.1748 - accuracy: 0.9552 - val_loss: 1.6698 - val_accuracy: 0.5402
Epoch 61/1000
201/201 [==============================] - 0s 141us/step - loss: 0.1702 - accuracy: 0.9453 - val_loss: 1.6982 - val_accuracy: 0.5287
Epoch 62/1000
201/201 [==============================] - 0s 137us/step - loss: 0.1741 - accuracy: 0.9303 - val_loss: 1.6255 - val_ac

Epoch 112/1000
201/201 [==============================] - 0s 144us/step - loss: 0.0970 - accuracy: 0.9652 - val_loss: 2.0783 - val_accuracy: 0.5747
Epoch 113/1000
201/201 [==============================] - 0s 137us/step - loss: 0.0899 - accuracy: 0.9751 - val_loss: 2.1140 - val_accuracy: 0.5862
Epoch 114/1000
201/201 [==============================] - 0s 129us/step - loss: 0.0930 - accuracy: 0.9751 - val_loss: 2.1103 - val_accuracy: 0.5632
Epoch 115/1000
201/201 [==============================] - 0s 138us/step - loss: 0.1025 - accuracy: 0.9701 - val_loss: 2.2794 - val_accuracy: 0.5977
Epoch 116/1000
201/201 [==============================] - 0s 138us/step - loss: 0.1108 - accuracy: 0.9652 - val_loss: 2.0904 - val_accuracy: 0.5862
Epoch 117/1000
201/201 [==============================] - 0s 134us/step - loss: 0.1098 - accuracy: 0.9502 - val_loss: 2.1177 - val_accuracy: 0.5747
Epoch 118/1000
201/201 [==============================] - 0s 130us/step - loss: 0.0925 - accuracy: 0.9652 - val_

201/201 [==============================] - 0s 134us/step - loss: 0.0686 - accuracy: 0.9751 - val_loss: 2.4304 - val_accuracy: 0.5862
Epoch 168/1000
201/201 [==============================] - 0s 131us/step - loss: 0.0605 - accuracy: 0.9751 - val_loss: 2.2900 - val_accuracy: 0.5977
Epoch 169/1000
201/201 [==============================] - 0s 147us/step - loss: 0.0642 - accuracy: 0.9751 - val_loss: 2.3882 - val_accuracy: 0.5977
Epoch 170/1000
201/201 [==============================] - 0s 167us/step - loss: 0.0649 - accuracy: 0.9801 - val_loss: 2.3878 - val_accuracy: 0.5862
Epoch 171/1000
201/201 [==============================] - 0s 132us/step - loss: 0.0638 - accuracy: 0.9801 - val_loss: 2.4519 - val_accuracy: 0.5977
Epoch 172/1000
201/201 [==============================] - 0s 136us/step - loss: 0.0775 - accuracy: 0.9652 - val_loss: 2.3472 - val_accuracy: 0.5977
Epoch 173/1000
201/201 [==============================] - 0s 162us/step - loss: 0.0691 - accuracy: 0.9701 - val_loss: 2.3800 - 

201/201 [==============================] - 0s 144us/step - loss: 0.0526 - accuracy: 0.9652 - val_loss: 2.8775 - val_accuracy: 0.5517
Epoch 223/1000
201/201 [==============================] - 0s 151us/step - loss: 0.0638 - accuracy: 0.9801 - val_loss: 2.6338 - val_accuracy: 0.5517
Epoch 224/1000
201/201 [==============================] - 0s 159us/step - loss: 0.0501 - accuracy: 0.9801 - val_loss: 2.8014 - val_accuracy: 0.5517
Epoch 225/1000
201/201 [==============================] - 0s 141us/step - loss: 0.0440 - accuracy: 0.9900 - val_loss: 2.7364 - val_accuracy: 0.5632
Epoch 226/1000
201/201 [==============================] - 0s 149us/step - loss: 0.0545 - accuracy: 0.9801 - val_loss: 2.9123 - val_accuracy: 0.5517
Epoch 227/1000
201/201 [==============================] - 0s 154us/step - loss: 0.0494 - accuracy: 0.9851 - val_loss: 2.7380 - val_accuracy: 0.5632
Epoch 228/1000
201/201 [==============================] - 0s 140us/step - loss: 0.0838 - accuracy: 0.9652 - val_loss: 2.8322 - 

...

201/201 [==============================] - 0s 149us/step - loss: 0.0247 - accuracy: 0.9900 - val_loss: 5.0055 - val_accuracy: 0.6092
Epoch 773/1000
201/201 [==============================] - 0s 139us/step - loss: 0.0313 - accuracy: 0.9900 - val_loss: 5.0164 - val_accuracy: 0.5977
Epoch 774/1000
201/201 [==============================] - 0s 147us/step - loss: 0.0716 - accuracy: 0.9851 - val_loss: 4.4337 - val_accuracy: 0.5862
Epoch 775/1000
201/201 [==============================] - 0s 139us/step - loss: 0.0761 - accuracy: 0.9801 - val_loss: 4.6218 - val_accuracy: 0.5862
Epoch 776/1000
201/201 [==============================] - 0s 139us/step - loss: 0.0848 - accuracy: 0.9602 - val_loss: 4.8105 - val_accuracy: 0.5977
Epoch 777/1000
201/201 [==============================] - 0s 130us/step - loss: 0.2641 - accuracy: 0.9403 - val_loss: 3.9673 - val_accuracy: 0.5517
Epoch 778/1000
201/201 [==============================] - 0s 125us/step - loss: 0.0712 - accuracy: 0.9602 - val_loss: 3.8155 - 

201/201 [==============================] - 0s 135us/step - loss: 0.0569 - accuracy: 0.9801 - val_loss: 4.2876 - val_accuracy: 0.5977
Epoch 828/1000
201/201 [==============================] - 0s 146us/step - loss: 0.0239 - accuracy: 0.9851 - val_loss: 4.1783 - val_accuracy: 0.5977
Epoch 829/1000
201/201 [==============================] - 0s 143us/step - loss: 0.0302 - accuracy: 0.9900 - val_loss: 4.1706 - val_accuracy: 0.6092
Epoch 830/1000
201/201 [==============================] - 0s 131us/step - loss: 0.0227 - accuracy: 0.9900 - val_loss: 4.2622 - val_accuracy: 0.5977
Epoch 831/1000
201/201 [==============================] - 0s 133us/step - loss: 0.0248 - accuracy: 0.9900 - val_loss: 4.2941 - val_accuracy: 0.5977
Epoch 832/1000
201/201 [==============================] - 0s 134us/step - loss: 0.0237 - accuracy: 0.9900 - val_loss: 4.2716 - val_accuracy: 0.5977
Epoch 833/1000
201/201 [==============================] - 0s 131us/step - loss: 0.0251 - accuracy: 0.9851 - val_loss: 4.3419 - 

201/201 [==============================] - 0s 140us/step - loss: 0.0299 - accuracy: 0.9851 - val_loss: 4.5809 - val_accuracy: 0.6092
Epoch 883/1000
201/201 [==============================] - 0s 149us/step - loss: 0.0263 - accuracy: 0.9851 - val_loss: 4.5238 - val_accuracy: 0.5977
Epoch 884/1000
201/201 [==============================] - 0s 145us/step - loss: 0.0267 - accuracy: 0.9900 - val_loss: 4.5505 - val_accuracy: 0.5977
Epoch 885/1000
201/201 [==============================] - 0s 131us/step - loss: 0.0241 - accuracy: 0.9851 - val_loss: 4.5925 - val_accuracy: 0.6092
Epoch 886/1000
201/201 [==============================] - 0s 132us/step - loss: 0.0241 - accuracy: 0.9851 - val_loss: 4.5741 - val_accuracy: 0.6092
Epoch 887/1000
201/201 [==============================] - 0s 138us/step - loss: 0.0228 - accuracy: 0.9851 - val_loss: 4.6004 - val_accuracy: 0.6092
Epoch 888/1000
201/201 [==============================] - 0s 135us/step - loss: 0.0234 - accuracy: 0.9900 - val_loss: 4.5886 - 

201/201 [==============================] - 0s 135us/step - loss: 0.0336 - accuracy: 0.9851 - val_loss: 4.6143 - val_accuracy: 0.5977
Epoch 938/1000
201/201 [==============================] - 0s 145us/step - loss: 0.0387 - accuracy: 0.9801 - val_loss: 4.4676 - val_accuracy: 0.5862
Epoch 939/1000
201/201 [==============================] - 0s 161us/step - loss: 0.0338 - accuracy: 0.9851 - val_loss: 4.5781 - val_accuracy: 0.5747
Epoch 940/1000
201/201 [==============================] - 0s 138us/step - loss: 0.0259 - accuracy: 0.9851 - val_loss: 4.5712 - val_accuracy: 0.5977
Epoch 941/1000
201/201 [==============================] - 0s 138us/step - loss: 0.0248 - accuracy: 0.9851 - val_loss: 4.4959 - val_accuracy: 0.6092
Epoch 942/1000
201/201 [==============================] - 0s 135us/step - loss: 0.0226 - accuracy: 0.9851 - val_loss: 4.5906 - val_accuracy: 0.5977
Epoch 943/1000
201/201 [==============================] - 0s 142us/step - loss: 0.0245 - accuracy: 0.9900 - val_loss: 4.6037 - 

201/201 [==============================] - 0s 137us/step - loss: 0.0212 - accuracy: 0.9900 - val_loss: 4.7852 - val_accuracy: 0.6092
Epoch 993/1000
201/201 [==============================] - 0s 169us/step - loss: 0.0249 - accuracy: 0.9900 - val_loss: 4.8049 - val_accuracy: 0.6092
Epoch 994/1000
201/201 [==============================] - 0s 174us/step - loss: 0.0215 - accuracy: 0.9900 - val_loss: 4.8375 - val_accuracy: 0.5862
Epoch 995/1000
201/201 [==============================] - 0s 193us/step - loss: 0.0215 - accuracy: 0.9900 - val_loss: 4.8088 - val_accuracy: 0.6092
Epoch 996/1000
201/201 [==============================] - 0s 170us/step - loss: 0.0209 - accuracy: 0.9900 - val_loss: 4.8186 - val_accuracy: 0.6092
Epoch 997/1000
201/201 [==============================] - 0s 189us/step - loss: 0.0240 - accuracy: 0.9900 - val_loss: 4.8315 - val_accuracy: 0.6092
Epoch 998/1000
201/201 [==============================] - 0s 178us/step - loss: 0.0217 - accuracy: 0.9851 - val_loss: 4.8306 - 

In [32]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

87/87 [==============================] - 0s 89us/step
over-sampling test accuracy: 60.92%


In [33]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [34]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 64 samples, validate on 28 samples
Epoch 1/100
64/64 [==============================] - 0s 1ms/step - loss: 1.2961 - accuracy: 0.3438 - val_loss: 0.9651 - val_accuracy: 0.5714
Epoch 2/100
64/64 [==============================] - 0s 154us/step - loss: 0.8636 - accuracy: 0.7188 - val_loss: 0.8267 - val_accuracy: 0.7143
Epoch 3/100
64/64 [==============================] - 0s 200us/step - loss: 0.7022 - accuracy: 0.8281 - val_loss: 0.7585 - val_accuracy: 0.8571
Epoch 4/100
64/64 [==============================] - 0s 349us/step - loss: 0.6216 - accuracy: 0.8125 - val_loss: 0.6923 - val_accuracy: 0.8571
Epoch 5/100
64/64 [==============================] - 0s 241us/step - loss: 0.5641 - accuracy: 0.8125 - val_loss: 0.6329 - val_accuracy: 0.8571
Epoch 6/100
64/64 [==============================] - 0s 242us/step - loss: 0.4984 - accuracy: 0.8594 - val_loss: 0.5672 - val_accuracy: 0.8571
Epoch 7/100
64/64 [==============================] - 0s 244us/step - loss: 0.4395 - accuracy: 0.8750

64/64 [==============================] - 0s 285us/step - loss: 0.0611 - accuracy: 1.0000 - val_loss: 0.1383 - val_accuracy: 0.9643
Epoch 58/100
64/64 [==============================] - 0s 220us/step - loss: 0.0593 - accuracy: 1.0000 - val_loss: 0.1374 - val_accuracy: 0.9643
Epoch 59/100
64/64 [==============================] - 0s 274us/step - loss: 0.0579 - accuracy: 1.0000 - val_loss: 0.1353 - val_accuracy: 0.9643
Epoch 60/100
64/64 [==============================] - 0s 264us/step - loss: 0.0564 - accuracy: 1.0000 - val_loss: 0.1334 - val_accuracy: 0.9643
Epoch 61/100
64/64 [==============================] - 0s 165us/step - loss: 0.0554 - accuracy: 1.0000 - val_loss: 0.1315 - val_accuracy: 0.9643
Epoch 62/100
64/64 [==============================] - 0s 195us/step - loss: 0.0547 - accuracy: 1.0000 - val_loss: 0.1301 - val_accuracy: 0.9643
Epoch 63/100
64/64 [==============================] - 0s 138us/step - loss: 0.0534 - accuracy: 1.0000 - val_loss: 0.1278 - val_accuracy: 0.9643
Epoch

In [36]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

28/28 [==============================] - 0s 248us/step
combination test accuracy: 96.43%


In [45]:
#### add dropout and regularizor
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [46]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 64 samples, validate on 28 samples
Epoch 1/100
64/64 [==============================] - 0s 2ms/step - loss: 4.4523 - accuracy: 0.3750 - val_loss: 1.0632 - val_accuracy: 0.6786
Epoch 2/100
64/64 [==============================] - 0s 147us/step - loss: 3.6197 - accuracy: 0.6875 - val_loss: 0.9041 - val_accuracy: 0.8571
Epoch 3/100
64/64 [==============================] - 0s 203us/step - loss: 4.4217 - accuracy: 0.6250 - val_loss: 0.8019 - val_accuracy: 0.8571
Epoch 4/100
64/64 [==============================] - 0s 183us/step - loss: 4.9893 - accuracy: 0.6094 - val_loss: 0.7419 - val_accuracy: 0.8571
Epoch 5/100
64/64 [==============================] - 0s 229us/step - loss: 2.2483 - accuracy: 0.7969 - val_loss: 0.7020 - val_accuracy: 0.8571
Epoch 6/100
64/64 [==============================] - 0s 261us/step - loss: 3.4671 - accuracy: 0.6719 - val_loss: 0.6705 - val_accuracy: 0.8571
Epoch 7/100
64/64 [==============================] - 0s 221us/step - loss: 2.6758 - accuracy: 0.7656

Epoch 58/100
64/64 [==============================] - 0s 243us/step - loss: 2.6612 - accuracy: 0.6875 - val_loss: 0.7569 - val_accuracy: 0.8571
Epoch 59/100
64/64 [==============================] - 0s 269us/step - loss: 1.7310 - accuracy: 0.7812 - val_loss: 0.7431 - val_accuracy: 0.8571
Epoch 60/100
64/64 [==============================] - 0s 234us/step - loss: 1.4096 - accuracy: 0.8750 - val_loss: 0.7373 - val_accuracy: 0.8571
Epoch 61/100
64/64 [==============================] - 0s 140us/step - loss: 2.5387 - accuracy: 0.6875 - val_loss: 0.7411 - val_accuracy: 0.8571
Epoch 62/100
64/64 [==============================] - 0s 216us/step - loss: 1.5868 - accuracy: 0.7812 - val_loss: 0.7414 - val_accuracy: 0.8571
Epoch 63/100
64/64 [==============================] - 0s 213us/step - loss: 1.9769 - accuracy: 0.6719 - val_loss: 0.7426 - val_accuracy: 0.8571
Epoch 64/100
64/64 [==============================] - 0s 156us/step - loss: 2.0616 - accuracy: 0.7812 - val_loss: 0.7412 - val_accuracy:

In [48]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

28/28 [==============================] - 0s 114us/step
combination test accuracy: 96.43%


In [49]:
############## Random Forest ##############

In [50]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 55.17%


In [56]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [57]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [54]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.54761905 0.61904762 0.58974359 0.61538462 0.66666667]
0.6076923076923076


In [55]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         0.92307692 1.         1.         1.        ]
0.9846153846153847
